# Contact area detection

## Coordinate creation

In [1]:
import numpy as np

In [2]:
check_collinear = lambda p1, p2, p3: np.linalg.norm(np.cross((p2-p1), (p1-p3))) 

In [3]:
dcv = lambda x, y, z: np.array([x / np.sqrt(x**2 + y**2 + z**2),
                                y / np.sqrt(x**2 + y**2 + z**2),
                                z / np.sqrt(x**2 + y**2 + z**2)])

In [4]:
def coordinate_dcm(origin,p1,p2):
    
    if abs(check_collinear(origin,p1,p2)) < 1.0e-6:
        
        print("Points are collinear, Select largest line and create a kp")
        
    else:
        
        v1= p1- origin                 #vector-1
        v2= p2- origin                 #vector-2
        v1_dc=dcv(v1[0],v1[1],v1[2])
        v2_dc=dcv(v2[0],v2[1],v2[2])

        v3=np.cross(v1_dc, v2_dc)      #vector-3, perp to 1,2
        v3_dc=dcv(v3[0],v3[1],v3[2])  

        v4=np.cross(v3_dc, v1_dc)      #vector-4(y) perp vector-3(z) perp to vector-1(x)
        v4_dc=dcv(v4[0],v4[1],v4[2])

        return v1_dc, v4_dc, v3_dc

In [5]:
def dcm2angleZXY(R):
    
    Ry = np.arctan2(-R[0, 2], R[2, 2])
    Rx = np.arcsin(R[1, 2])
    Rz = np.arctan2(-R[1, 0], R[1, 1])
    
    return Rx, Ry ,Rz

**Example**

In [6]:
origin=np.array([10,50,10])
p1=np.array([10,10,11])
p2=np.array([5,5,15])

if check_collinear(origin,p1,p2)==0:
    print("Points are collinear, Select largest line and create a kp")
    
else:
    xl,yl,zl=coordinate_dcm(origin,p1,p2)
    print(xl)
    print(yl)
    print(zl)

[ 0.         -0.99968765  0.02499219]
[-0.7905077   0.01530652  0.6122608 ]
[-0.6124521  -0.01975652 -0.79026078]


In [7]:
R = np.array([[xl[0], yl[0], zl[0]],
              [xl[1], yl[1], zl[1]],
              [xl[2], yl[2], zl[2]]])
R

array([[ 0.        , -0.7905077 , -0.6124521 ],
       [-0.99968765,  0.01530652, -0.01975652],
       [ 0.02499219,  0.6122608 , -0.79026078]])

In [8]:
R=np.column_stack((xl, yl, zl))
R

array([[ 0.        , -0.7905077 , -0.6124521 ],
       [-0.99968765,  0.01530652, -0.01975652],
       [ 0.02499219,  0.6122608 , -0.79026078]])

In [9]:
# R = np.array([[xl[0], yl[0], zl[0]],
#               [xl[1], yl[1], zl[1]],
#               [xl[2], yl[2], zl[2]]])

R=np.column_stack((xl, yl, zl))

angles = dcm2angleZXY(R)
print(angles)

(-0.01975780494578869, 2.4822825852569355, 1.555486220526841)


## Check 4th point in plane

In [10]:
check_coplaner = lambda p1, p2, p3, p4: np.linalg.det(np.array([ [p1[0], p2[0], p3[0], p4[0]],
                                                                  [p1[1], p2[1], p3[1], p4[1]],
                                                                  [p1[2], p2[2], p3[2], p4[2]],
                                                                  [1,     1,      1,     1  ] ]))



**Example**

In [11]:
p1= np.array([0, 0, 0])
p2= np.array([1, 0, 0])
p3= np.array([0, 1, 0])
p4= np.array([0, 0, 0])


if abs(check_coplaner(p1, p2, p3, p4)) < 1.0e-6:
    print("PLANE")
else:
    print("Not plane")


PLANE


## Find center of circle from three point 

https://stackoverflow.com/questions/20314306/find-arc-circle-equation-given-three-points-in-space-3d

https://mathworld.wolfram.com/BarycentricCoordinates.html

https://mathworld.wolfram.com/Circumcenter.html

https://mathworld.wolfram.com/Circumcircle.html

https://github.com/sergarrido/random/tree/master/circle3d

https://new.math.uiuc.edu/public403/affine/bary.html

In [12]:
import numpy as np

def circle_center_radius(A, B, C):

    if abs(check_collinear(A,B,C)) < 1.0e-6:      
        raise ValueError("Points are collinear, no unique circle exists.")
    
    a = np.linalg.norm(C - B)
    b = np.linalg.norm(C - A)
    c = np.linalg.norm(B - A)
    
    s = (a + b + c) / 2
    rad = a*b*c / 4 / np.sqrt(s * (s - a) * (s - b) * (s - c))
    
     #Barycentric Coordinates of circumsnter
    b1 = a*a * (b*b + c*c - a*a)
    b2 = b*b * (a*a + c*c - b*b)
    b3 = c*c * (a*a + b*b - c*c)     
    
     #Barycentric Coordinates to cartesian Coordinates 
    centr = np.column_stack((A, B, C)).dot(np.hstack((b1, b2, b3)))  
    centr /= b1 + b2 + b3
#     centr = centr/(b1 + b2 + b3)
    
    return centr,rad.dot(np.hstack((b1, b2, b3)))  
    centr /= b1 + b2 + b3
#     centr = centr/(b1 + b2 + b3)
    
    return centr,rad


In [13]:
import numpy as np
def circle_center_radius(A, B, C):

    if abs(check_collinear(A,B,C)) < 1.0e-6:      
        raise ValueError("Points are collinear, no unique circle exists.")
    
    a = np.linalg.norm(C - B)
    b = np.linalg.norm(C - A)
    c = np.linalg.norm(B - A)
    
    s = (a + b + c) / 2
    rad = a*b*c / 4 / np.sqrt(s * (s - a) * (s - b) * (s - c))
    
     #Barycentric Coordinates of circumsnter
    b1 = a*a * (b*b + c*c - a*a)
    b2 = b*b * (a*a + c*c - b*b)
    b3 = c*c * (a*a + b*b - c*c)     
    
     #Barycentric Coordinates to cartesian Coordinates 
    centr = np.column_stack((A, B, C)).dot(np.hstack((b1, b2, b3)))  
    centr /= b1 + b2 + b3
#     centr = centr/(b1 + b2 + b3)
    
    return centr,rad

**Example**

In [14]:
point1 = np.array([1, 1, 0])
point2 = np.array([1, -1, 0])
point3 = np.array([-1, 1, 0])

Center,Radius=circle_center_radius(point1, point2, point3)
print(Center)
print(Radius)

# point1.dot((2,2,2))

[-2.22044605e-16 -2.22044605e-16  0.00000000e+00]
1.4142135623730958


In [15]:
# Points are collinear

point1 = np.array([1, 2, 3])
point2 = np.array([4, 5, 6])
point3 = np.array([7, 8, 9])

Center,Radius=circle_center_radius(point1, point2, point3)
print(Center)
print(Radius)

ValueError: Points are collinear, no unique circle exists.

## Check three vector normality

In [23]:
check_normal = lambda v1, v2, v3: np.abs(np.array([np.dot(v1,v2), np.dot(v2,v3), np.dot(v3,v1)])) < np.array([1.0e-6,1.0e-6,1.0e-6])

check_normal.__doc__ = """Check if all three vecors are perpendicular to each other, return false for not perpendicular pair"""

In [24]:
# check_normal = lambda v1, v2, v3: np.abs(np.array([np.dot(v1,v2), 
#                                                  np.dot(v2,v3), 
#                                                  np.dot(v3,v1)])) < np.array([1.0e-6,
#                                                                               1.0e-6,
#                                                                               1.0e-6])

In [25]:
print(check_normal.__doc__)

Check if all three vecors are perpendicular to each other, return false for not perpendicular pair
